In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

# Packages

In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import datetime
from datetime import datetime, timedelta
#import plotly.graph_objs as go
import matplotlib.pyplot as plt
import csv
import seaborn as sns
%matplotlib inline
sns.set()
from PIL import Image

# Confirmed dengue dataset

In [3]:
df = pd.read_csv('/Users/julianeoliveira/Desktop/Modelling TaskForce Arboviruses/Data/dengue_conf_clean.csv',low_memory=False)

In [4]:
df['dt_sin_pri'] = pd.to_datetime(df['dt_sin_pri'], errors = 'coerce')
df['dt_notific'] = pd.to_datetime(df['dt_notific'])
df['dt_nasc'] = pd.to_datetime(df['dt_nasc'])

In [5]:
df['sem_pri_correct'] = df['dt_sin_pri'].dt.strftime('%Y-%W')

In [6]:
df['month'] = df['dt_sin_pri'].dt.strftime('%Y-%m')

## Series of confirmed dengue cases by week and municipality

In [7]:
data = df.filter(['codmunres','dt_sin_pri','year','month','sem_pri','sem_pri_correct','cs_sexo','faixa','count'])

In [8]:
dados = data.groupby(['codmunres','year','month','sem_pri_correct','cs_sexo','faixa'])['count'].sum().reset_index()

In [9]:
dados.head(30)

,codmunres,year,month,sem_pri_correct,cs_sexo,faixa,count
0,110000.0,2015,2015-12,2015-52,1,4,1
1,110001.0,2010,2010-01,2010-02,2,7074,1
2,110001.0,2010,2010-01,2010-03,2,5559,1
3,110001.0,2010,2010-02,2010-06,1,1014,2
4,110001.0,2010,2010-02,2010-06,1,3539,1
5,110001.0,2010,2010-02,2010-06,2,6064,1
6,110001.0,2010,2010-02,2010-07,1,509,1
7,110001.0,2010,2010-02,2010-07,1,5054,1
8,110001.0,2010,2010-02,2010-07,2,2024,1
9,110001.0,2010,2010-02,2010-08,1,4549,1


## Some descritives checks

In [10]:
dados.groupby(['year'])['count'].sum()

year
2010     869202
2011     600937
2012     401708
2013    1188781
2014     475207
2015    1386871
2016     479362
2017     167600
2018     146490
2019    1280783
Name: count, dtype: int64

In [11]:
df.year.value_counts()

2015    1387045
2019    1280791
2013    1188923
2010     869246
2011     601017
2016     479382
2014     475282
2012     401768
2017     167604
2018     146494
Name: year, dtype: int64

# Population data to calculate the risk

In [12]:
pop = pd.read_csv('/Users/julianeoliveira/Desktop/Modelling TaskForce Arboviruses/Data/pop.csv')

In [13]:
pop = pop.astype(int)

# Standard incidence ratio of dengue

## Expected number of cases per week and municipality

In [42]:
teste_pop = pop.groupby(['MUNIC_RES','ANO','SEXO','FXETARIA'])['POPULACAO'].sum().reset_index()
teste_pop.head()

,MUNIC_RES,ANO,SEXO,FXETARIA,POPULACAO
0,110001,2012,1,4,943
1,110001,2012,1,509,1058
2,110001,2012,1,1014,1239
3,110001,2012,1,1519,1343
4,110001,2012,1,2024,1090


In [43]:
table_pop= teste_pop.groupby(['SEXO','FXETARIA'])['POPULACAO'].sum().reset_index()

In [63]:
teste = dados

In [64]:
teste = teste[teste.cs_sexo != 'I']

In [65]:
teste.codmunres = teste['codmunres'].astype(int)
#teste = teste.astype(int)

/Users/julianeoliveira/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Let $j$ denoting a combination of age and gender categories (strata). If $j$ has $O_{ij}$ observed events out of a population $Pop_{ij}$, then expected number of cases $E_i$ for an area, $ i$, represents the prior spatial distribution of the disease. This distribution is established by the population structure, which varies according to the combination of age and gender categories of the whole population. In such a way, the expected number of events in this strata is given by:
\begin{equation}\label{eq1}
E_{ij} = Pop_{ij}\frac{\sum_{i=1}^{n}O_{ij}}{\sum_{i=1}^{n}Pop_{ij}},
\end{equation}
and the total number of expected events in area $i$	 is just:
\begin{equation}\label{eq2}
E_{i} = \sum_{j=1}^{J}E_{ij},
\end{equation}
where $n$ is the number of aggregated count and $J$ the number of categories \cite{Leyland2005}. 

	The relative risk within an area is given as the ratio between the observed and expected number of cases of the disease. This ratio is the so called standard incidence ratio and is given by:
\begin{equation}\label{eq3}
\theta_i = \frac{O_{i}}{E_{i}}.
\end{equation}
If the observed number of cases equals the expected number, the $\theta$ is 1. If more cases are observed than expected, then $\theta$ is greater than 1. If fewer cases are observed than expected, the $\theta$ is less than 1. 


In [66]:
teste.head()

,codmunres,year,month,sem_pri_correct,cs_sexo,faixa,count
0,110000,2015,2015-12,2015-52,1,4,1
1,110001,2010,2010-01,2010-02,2,7074,1
2,110001,2010,2010-01,2010-03,2,5559,1
3,110001,2010,2010-02,2010-06,1,1014,2
4,110001,2010,2010-02,2010-06,1,3539,1


In [ ]:
SOij_Spij_month = []
for month in teste.month:
    
    #df_month = teste[teste.month == month]
    #table_month = pd.pivot_table(df_month, values='count', index=['codmunres'], columns=['cs_sexo','faixa']).fillna(0)
    #resulted_dataframe = table_year.sum().reset_index()
    
    datamonth = teste[teste.month == month].groupby(['cs_sexo','faixa'])['count'].sum().reset_index()
    
    datamonth['SOij_Spij'] = datamonth['count']/table_pop.POPULACAO
    
    SOij_Spij_month.append(datamonth)
    
    

In [58]:
SOij_Spij_month

[]

In [40]:
dataweek1['Eij'] =popij_muni1['POPULACAO']*dataweek1['SOij_Spij']

In [41]:
sum(dataweek1['Eij'])

0.8255987627217654

In [38]:
popij_muni1 = teste_pop[teste_pop['MUNIC_RES'] == 110001]

In [ ]:
sum_observed_gender_age_week = []
for week in teste.sem_pri_correct:
    df_week = teste[teste.sem_pri_correct == week].groupby(['cs_sexo','faixa'])['count'].sum().reset_index()
    sum_observed_gender_age_week.append(df_week)